# Chapter 12. Advanced pandas

In [1]:
import pandas as pd
import numpy as np

## 12.1 Categorical Data
### Background and Motivation

In [2]:
values=pd.Series(['apple','orange','apple','apple']*2)

In [3]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

**A best practice is to use so-called *dimension tables* containing distinct values and storing the primary observations as integer keys referencing the dimension table**

In [6]:
values=pd.Series([0,1,0,0]*2)

In [11]:
dim=pd.Series(['apple','orange'])

In [8]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

**Use the** take **method to restore the original Series of Strings**<br>
* **The array of distinct values can be called *categoties,dictionary,or levels* of the data**
* **The integer values that reference the categories are called *category codes***

In [12]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Type in pandas

In [13]:
fruits=['apple','orange','apple','apple']*2

In [14]:
N=len(fruits)

In [19]:
df=pd.DataFrame({'fruit':fruits,
                 'basked_id':np.arange(N),
                 'count':np.random.randint(3,15,size=N),
                 'weight':np.random.uniform(0,4,size=N)},
                 columns=['basket_id','fruit','count','weight'])

In [20]:
df

,basket_id,fruit,count,weight
0,NaN,apple,12,3.450648
1,NaN,orange,10,1.982212
2,NaN,apple,4,1.204551
3,NaN,apple,4,0.333902
4,NaN,apple,8,3.094731
5,NaN,orange,10,2.502602
6,NaN,apple,13,3.635739
7,NaN,apple,10,1.043563


In [21]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

**Convert a Python string object to categorical by using** astype()

In [39]:
fruit_cat=df['fruit'].astype('category')

In [43]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [36]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

**The values for** fruit_cat **are not a Numpy array, but an instances of** pandas.Categorical

In [46]:
c=fruit_cat.values

In [47]:
type(c)

pandas.core.arrays.categorical.Categorical

Categorical **object has** categories **and** codes **attributes**

In [50]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [51]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

**You can convert a DataFrame to categorical by assigning the converted result**

In [53]:
df['fruit']=df['fruit'].astype('category')

In [54]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

**You can also created** pandas.Categorical **directly from other types of Python sequences**

In [56]:
my_categories=pd.Categorical(['foo','bar','baz','foo','bar'])

In [57]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

**If you have obtained categorical encoded data from another source, you can use the alternative** from_codes **constructor**

In [62]:
categories=['foo','bar','baz']

In [63]:
codes=[0,1,2,0,0,1]

In [65]:
my_cats_2=pd.Categorical.from_codes(codes,categories)

In [66]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

**Unless explicitly specified, categorical conversions assume no specific ordering of the categories**<br>
**When using** from_codes **or any of the other constructors, you can indicate that the categories have  a meaningful ordering**

In [67]:
ordered_cat=pd.Categorical.from_codes(codes,categories,ordered=True)

In [68]:
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

**An unordered categorical instance can be made ordered with** as_ordered

In [69]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

**A categorical array can consist of any immutable value types**

### Computation with Categoricals

**Some parts of pandas, like the** groupby **function, perform better when working with categoricals**

In [70]:
np.random.seed(12345)

In [71]:
draws=np.random.randn(1000)

In [72]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

**Compute a quartile binning of the data and extract some statistics**

In [73]:
bins=pd.qcut(draws,4)

In [74]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

**Add quartile names using** lables **argument to** qcut

In [75]:
bins=pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])

In [76]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [77]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

**We can use** groupby **to extract some summary statistics**

In [78]:
bins=pd.Series(bins,name='quartile')

In [79]:
results=(pd.Series(draws)
        .groupby(bins)
        .agg(['count','min','max'])
        .reset_index())

In [80]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### Better performance with categoricals
**A categorical version of a DataFrame column will often use significantly less memory**

In [81]:
N=10000000
draws=pd.Series(np.random.randn(N))

In [84]:
labels=pd.Series(['foo','bar','baz','quz']*(N//4))

In [86]:
categories=labels.astype('category')

In [87]:
labels.memory_usage()

80000128

In [89]:
categories.memory_usage()

10000320

**GroupBy operations can be significantly faster with categorical because the underlying algorithm use the integer based codes array insetad of an array of strings**

### Categorical Methods
**Series containing categorical data have several special methods similar to the** Series.str **specialized methods**

In [91]:
s=pd.Series(['a','b','c','d']*2)

In [92]:
cat_s=s.astype('category')

**The special attribute** cat **provides access to categorical methods**

In [95]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [96]:
actual_categories=['a','b','c','d','e']

In [98]:
cat_s2=cat_s.cat.set_categories(actual_categories)

In [99]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

**While it appears unchanged, the new categories will be reflected in operation that uses them**

In [100]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [101]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

* **Categoricals are often used as a convenient tool for memory and better performance**
* **We can use the** remove_unused_categories **method to trim unobserved categories**

In [102]:
cat_s3=cat_s[cat_s.isin(['a','b'])]

In [103]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [104]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

### Creating dummy variables for modeling
***One-hot* encoding: Transform categorical data into *dummy variables***<br>
**Use** pandas.get_dummies **function**

In [105]:
cat_s=pd.Series(['a','b','c','d']*2,dtype='category')

In [106]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 Advanced GroupBy Use
### Group Transforms and "Unwrapped" GroupBys
* **There is another built-in method called** transform, **which is similar to apply but impose more constraints on the kind of function you can use**
* **It can produce a scalar value to be broadcast to the shape of the group**
* **It can produce an object of the same shape as the input group**
* **It must not mutate its input**

In [107]:
df=pd.DataFrame({'key':['a','b','c']*4,
                'value':np.arange(12.)})

In [108]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


**Group means by key**

In [110]:
g=df.groupby('key').value

In [111]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

**Produce a Series of the same shape as** df\['Value'\] **but with values replaces by the average grouped by** 'key' 

In [113]:
g.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

**For built-in aggregation functions, we can pass a string alias with the GroupBy** agg **method**

In [115]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

**Like** apply, transform **works with functions that Series, but the result must be the same size as the input**

In [116]:
g.transform(lambda x:x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

**We can compute the ranks in descending order for each group**

In [117]:
g.transform(lambda x:x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

**Consider a group transformation function composed from a simple aggregations**

In [118]:
def normalize(x):
    return (x-x.mean())/x.std()

In [119]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

**Built-in aggregate functions like** 'mean' **or** 'sum' **are often much faster than a general** apply **function**<br>
* **These also have a "fast past" when used with** transform<br>
* **This allows us to perform a so-called** *unwrapped* **group operation**

In [120]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [121]:
normalized=(df['value']-g.transform('mean'))/g.transform('std')

In [122]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## Grouped Time Resampling
**For time series data, the** resample **method is semantically a group operation based on a time intervalization**

In [123]:
N=15

In [126]:
times=pd.date_range('2017-05-20 00:00',freq='1min',periods=N)

In [127]:
df=pd.DataFrame({'time':times,
                'value':np.arange(N)})

In [128]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


**We can index by** 'time' **and then resample**

In [129]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


**Suppose that a DataFrame contains multiple time series, marked by an additional** group **key column**

In [130]:
df2=pd.DataFrame({'time':times.repeat(3),
                 'key':np.tile(['a','b','c'],N),
                 'value':np.arange(N*3)})

In [131]:
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0
1,2017-05-20 00:00:00,b,1
2,2017-05-20 00:00:00,c,2
3,2017-05-20 00:01:00,a,3
4,2017-05-20 00:01:00,b,4
5,2017-05-20 00:01:00,c,5
6,2017-05-20 00:02:00,a,6


**To do the same resampling for each value of** 'key', **we introduce the** pandas.Grouper **object**

In [143]:
time_key=pd.Grouper('5min')

In [149]:
time_key

Grouper(key='5min', axis=0, sort=False)

## 12.3 Techniques for Method Chaining

DataFrame.assign **method is a functional alternative to column assignment of the form** df\[k\]=b<br>
* **Returns a new DataFrame with the indicated modications**

In [ ]:
#Usual non-functional way
df2=df.copy()
df2['k']=v
#Functional assign way
df2=df.assign(k=v)

pipe **method**

In [ ]:
a=f(df,arg1=v1)
b=g(a,v2,arg3=v3)
c=h(b,arg4=v4)

**When using functions that accept and return Series or DataFrame objects, you can rewrite this using calls to** pipe

In [ ]:
result=(df.pipe(f,arg1=v1)
       .pipe(g,v2,arg3=v3)
       .pipe(h,arg4=v4))

**Subtract group mean from a column**

In [ ]:
g=df.groupby(['key1','key1'])
df['col1']=df['col1']-g.transform('mean')

**Method chaining implementation**

In [ ]:
def group_demean(df,by,cols):
    result=df.copy
    g=df.groupby(by)
    for c in cols:
        result[c]=df[c]-g[c].transform('mean')

In [ ]:
result=(df[df.col1<0]
       .pipe(group_demean,['key1','key2'],['col1']))

In [151]:
%%bash
git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .ipynb_checkpoints/Untitled-checkpoint.ipynb
	modified:   Data_Loading_Practice.ipynb
	modified:   NumPy_Basics_Arrays_and_Vectorized_Computation.ipynb
	modified:   Untitled.ipynb
	modified:   Untitled1.ipynb
	modified:   data_cleaning.ipynb
	modified:   datawrangling.ipynb
	modified:   group_operation.ipynb
	modified:   plotting_and_visulization.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	.ipynb_checkpoints/Data_Loading_Practice-checkpoint.ipynb
	.ipynb_checkpoints/NumPy_Basics_Arrays_and_Vectorized_Computation-checkpoint.ipynb
	.ipynb_checkpoints/TimeSeries (1column)-checkpoint.ipynb
	.ipynb_checkpoints/Untitled2-checkpoint.ipynb
	.ipynb_checkpoints/advanced_pandas-checkpoint.ipynb
	.ipynb

In [ ]:
%%